In [ ]:
import pandas as pd
import random

In [68]:
origin_dataset = pd.read_excel('/content/total_data.xlsx')

In [ ]:
origin_dataset

In [70]:
## 임의로 고객 정보 생성; 보유 주식 종목, 희망 종가 상한
numlist = list(range(0, len(origin_dataset), 5))
sample_numlist = random.sample(numlist, random.randint(1, 10))
sample_list = []
for num in sample_numlist:
  for j in range(5):
    sample_list.append(num+j)
consumer_data = origin_dataset.iloc[sample_list] ##임의로 지정

consumer_price = random.randint(min(origin_dataset['종가']), max(origin_dataset['종가'])) ##임의로 지정

In [ ]:
consumer_data

In [ ]:
consumer_price

In [72]:
## 고객 보유 주식을 제외한 나머지 주식 정보
dataset = pd.merge(consumer_data, origin_dataset, how = 'outer', indicator = True).query('_merge == "right_only"').drop(columns=['_merge'])

In [ ]:
dataset

In [73]:
## 유사도 계산 알고리즘 (필터링 방식)
recommend_data = pd.DataFrame()
show_data = pd.DataFrame()

for i in range(len(consumer_data)//5):
  for k in range(len(dataset)//5):
    count = 0
    ## 업종 (같은 분야의 주식 종목만 필터링)
    if consumer_data.iloc[5*i]['업종명'] == dataset.iloc[5*k]['업종명']:

      ## 희망 종가 (희망 종가보다 가격이 낮은 주식 종목만 필터링)
      if consumer_price >= dataset.iloc[5*k+4]['종가']:

        ##등락률 (등락률 방향이 같은 주식 종목만 필터링)
        for l in range(5):    
          if consumer_data.iloc[5*i+l]['등락률']*dataset.iloc[5*k+l]['등락률'] > 0: 
            count += 1
      
        if count == 5:
          recommend_data = pd.concat([recommend_data, dataset.iloc[[5*k, 5*k+1, 5*k+2, 5*k+3, 5*k+4]]])
          show_data = pd.concat([show_data, dataset.iloc[5*k+4]], axis = 1)

recommend_data = recommend_data.dropna()
show_data = show_data.dropna()
show_data = show_data.transpose()      

In [ ]:
## 결과 제시 (종목명만 제시 / 마지막 날 정보만 제시 / 전체 정보 제시)
result_method = int(input('1:종목명만 제시, 2:마지막 날 정보만 제시, 3:전체 정보 제시 : '))
try:
  if result_method == 1:
    print(show_data['종목명'].tolist())
  elif result_method == 2:
    print(show_data)
  elif result_method == 3:
    print(recommend_data)
  
except:
  print('no recommend')